# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [47]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import pickle
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

import warnings

warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kaypa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kaypa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kaypa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [48]:
# load data from database
engine = create_engine('sqlite:///Messages.db')
df = pd.read_sql("SELECT * FROM Messages", engine)

X = df['message']
Y = df.drop(['index', 'message', 'original', 'genre'], axis = 1)

In [49]:
df

,index,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,2,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,4,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26243,26243,"""The pattern is always the same: steal, loot, ...",None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26244,26244,"Sirajganj, Bangladesh. When the rain-swollen J...",None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26245,26245,Damage to the Nacala rail corridor by the rain...,None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26246,26246,"Thus, an armed drone strike in Pakistan, a Nig...",None,news,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [32]:
def tokenize(text):
    """Normalize, tokenize and stem text string
    
    Args:
    text: string. String containing message for processing
       
    Returns:
    stemmed: list of strings. List containing normalized and stemmed word tokens
    """
    # Convert text to lowercase and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize words
    tokens = word_tokenize(text)
    
    # Stem word tokens and remove stop words
    stemmer = PorterStemmer()
    stop_words = stopwords.words("english")
    
    stemmed = [stemmer.stem(word) for word in tokens if word not in stop_words]
    
    return stemmed

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [33]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [34]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1)

np.random.seed(17)
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001F577E88820>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall on both the training set and the test set. You can use sklearn's `classification_report` function here. 

In [35]:
def get_eval_metrics(actual, predicted, col_names):
    """Calculate evaluation metrics for ML model
    
    Args:
    actual: array. Array containing actual labels.
    predicted: array. Array containing predicted labels.
    col_names: list of strings. List containing names for each of the predicted fields.
       
    Returns:
    metrics_df: dataframe. Dataframe containing the accuracy, precision, recall 
    and f1 score for a given set of actual and predicted labels.
    """
    metrics = []
    
    # Calculate evaluation metrics for each set of labels
    for i in range(len(col_names)):
        accuracy = accuracy_score(actual[:, i], predicted[:, i])
        precision = precision_score(actual[:, i], predicted[:, i], average='weighted')
        recall = recall_score(actual[:, i], predicted[:, i], average='weighted')
        f1 = f1_score(actual[:, i], predicted[:, i], average='weighted')
        
        metrics.append([accuracy, precision, recall, f1])
    
    # Create dataframe containing metrics
    metrics = np.array(metrics)
    metrics_df = pd.DataFrame(data = metrics, index = col_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
      
    return metrics_df    

In [36]:
# Calculate evaluation metrics for training set
Y_train_pred = pipeline.predict(X_train)
col_names = list(Y.columns.values)

print(get_eval_metrics(np.array(Y_train), Y_train_pred, col_names))

                        Accuracy  Precision    Recall        F1
related                 0.995327   0.995324  0.995327  0.995319
request                 0.995987   0.995981  0.995987  0.995979
offer                   0.999898   0.999898  0.999898  0.999898
aid_related             0.993904   0.993906  0.993904  0.993903
medical_help            0.998222   0.998223  0.998222  0.998214
medical_products        0.998019   0.998015  0.998019  0.998004
search_and_rescue       0.999746   0.999746  0.999746  0.999745
security                0.999644   0.999645  0.999644  0.999643
military                0.999644   0.999645  0.999644  0.999644
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.998324   0.998324  0.998324  0.998315
food                    0.997613   0.997612  0.997613  0.997604
shelter                 0.997359   0.997361  0.997359  0.997343
clothing                0.999594   0.999593  0.999594  0.999592
money                   0.999441   0.999

In [37]:
# Calculate evaluation metrics for test set
Y_test_pred = pipeline.predict(X_test)

eval_metrics0 = get_eval_metrics(np.array(Y_test), Y_test_pred, col_names)
print(eval_metrics0)

                        Accuracy  Precision    Recall        F1
related                 0.745809   0.654134  0.745809  0.666957
request                 0.826120   0.768725  0.826120  0.766090
offer                   0.996038   0.992394  0.996038  0.994213
aid_related             0.571167   0.539889  0.571167  0.530296
medical_help            0.922280   0.863030  0.922280  0.889645
medical_products        0.947120   0.910747  0.947120  0.923450
search_and_rescue       0.974855   0.950343  0.974855  0.962443
security                0.981865   0.964059  0.981865  0.972881
military                0.969521   0.939972  0.969521  0.954518
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.933252   0.886075  0.933252  0.905095
food                    0.885706   0.808963  0.885706  0.837570
shelter                 0.906126   0.836587  0.906126  0.868424
clothing                0.983237   0.975281  0.983237  0.975224
money                   0.976989   0.954

For the vast majority of the categories, the indicators Accuracy, Precision, Recall and F1 are very good. 

### 6. Improve your model
Use grid search to find better parameters. 

In [38]:
# Define performance metric for use in grid search scoring object
def performance_metric(y_true, y_pred):
    """Calculate median F1 score for all of the output classifiers
    
    Args:
    y_true: array. Array containing actual labels.
    y_pred: array. Array containing predicted labels.
        
    Returns:
    score: float. Median F1 score for all of the output classifiers
    """
    f1_list = []
    for i in range(np.shape(y_pred)[1]):
        f1 = f1_score(np.array(y_true)[:, i], y_pred[:, i])
        f1_list.append(f1)
        
    score = np.median(f1_list)
    return score

The following code is very handy to choose how to improve the model. 

In [50]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf_

I use 'clf__estimator__class_weight': ['balanced'] to obtain more diversified results. In other words, to avoid possible issues of unbalanced data.

In [53]:
# Create grid search object

parameters = {'clf__estimator__max_depth':[2, 5, 10],
              'clf__estimator__n_estimators':[5, 10], 
              'clf__estimator__min_samples_split':[2, 5],
             'clf__estimator__class_weight': ['balanced'] }

scorer = make_scorer(performance_metric)
cv = GridSearchCV(pipeline, param_grid = parameters, scoring = scorer, verbose = 2)

# Find best parameters
np.random.seed(81)
tuned_model = cv.fit(X_train, Y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END clf__estimator__class_weight=balanced, clf__estimator__max_depth=2, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=5; total time=  24.9s
[CV] END clf__estimator__class_weight=balanced, clf__estimator__max_depth=2, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=5; total time=  21.2s
[CV] END clf__estimator__class_weight=balanced, clf__estimator__max_depth=2, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=5; total time=  20.5s
[CV] END clf__estimator__class_weight=balanced, clf__estimator__max_depth=2, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=5; total time=  21.1s
[CV] END clf__estimator__class_weight=balanced, clf__estimator__max_depth=2, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=5; total time=  21.9s
[CV] END clf__estimator__class_weight=balanced, clf__estimator__max_depth=2, clf__estimator__min_samples_split=2, 

[CV] END clf__estimator__class_weight=balanced, clf__estimator__max_depth=10, clf__estimator__min_samples_split=5, clf__estimator__n_estimators=5; total time=  20.2s
[CV] END clf__estimator__class_weight=balanced, clf__estimator__max_depth=10, clf__estimator__min_samples_split=5, clf__estimator__n_estimators=5; total time=  20.4s
[CV] END clf__estimator__class_weight=balanced, clf__estimator__max_depth=10, clf__estimator__min_samples_split=5, clf__estimator__n_estimators=5; total time=  19.7s
[CV] END clf__estimator__class_weight=balanced, clf__estimator__max_depth=10, clf__estimator__min_samples_split=5, clf__estimator__n_estimators=5; total time=  20.2s
[CV] END clf__estimator__class_weight=balanced, clf__estimator__max_depth=10, clf__estimator__min_samples_split=5, clf__estimator__n_estimators=5; total time=  20.4s
[CV] END clf__estimator__class_weight=balanced, clf__estimator__max_depth=10, clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10; total time=  21.3s
[CV

In [40]:
# Get results of grid search
tuned_model.cv_results_

{'mean_fit_time': array([15.960957  , 16.74645953, 17.18199406, 17.00588093, 15.78674955,
        15.88934331]),
 'std_fit_time': array([0.23333287, 0.77601532, 0.83868129, 1.83795215, 0.19691544,
        0.33368533]),
 'mean_score_time': array([3.92424855, 4.05096126, 4.10611706, 3.84239254, 3.85023494,
        3.8075943 ]),
 'std_score_time': array([0.05927206, 0.22392025, 0.10671318, 0.09237727, 0.08514312,
        0.17822861]),
 'param_clf__estimator__max_depth': masked_array(data=[2, 2, 5, 5, 10, 10],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_clf__estimator__min_samples_split': masked_array(data=[2, 5, 2, 5, 2, 5],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_clf__estimator__n_estimators': masked_array(data=[5, 5, 5, 5, 5, 5],
              mask=[False, False, False, False, False, False],
        fill_value='?',
            

In [41]:
# Parameters for best mean test score
tuned_model.best_params_

{'clf__estimator__max_depth': 2,
 'clf__estimator__min_samples_split': 2,
 'clf__estimator__n_estimators': 5}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.

In [42]:
# Calculate evaluation metrics for test set
tuned_pred_test = tuned_model.predict(X_test)

eval_metrics1 = get_eval_metrics(np.array(Y_test), tuned_pred_test, col_names)

print(eval_metrics1)

                        Accuracy  Precision    Recall        F1
related                 0.758915   0.575952  0.758915  0.654895
request                 0.828254   0.686004  0.828254  0.750447
offer                   0.996190   0.992395  0.996190  0.994289
aid_related             0.590674   0.531100  0.590674  0.441091
medical_help            0.926547   0.858489  0.926547  0.891220
medical_products        0.948339   0.899347  0.948339  0.923193
search_and_rescue       0.974855   0.950343  0.974855  0.962443
security                0.981865   0.964059  0.981865  0.972881
military                0.969521   0.939972  0.969521  0.954518
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.936300   0.876658  0.936300  0.905498
food                    0.889820   0.791780  0.889820  0.837942
shelter                 0.912831   0.833261  0.912831  0.871233
clothing                0.983237   0.966755  0.983237  0.974926
money                   0.977141   0.954

In [43]:
# Get summary stats for first model
eval_metrics0.describe()

,Accuracy,Precision,Recall,F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.925641,0.890804,0.925641,0.901039
std,0.090547,0.105334,0.090547,0.107376
min,0.571167,0.539889,0.571167,0.530296
25%,0.913365,0.861855,0.913365,0.878255
50%,0.953749,0.916773,0.953749,0.932502
75%,0.982208,0.966685,0.982208,0.973467
max,1.000000,1.000000,1.000000,1.000000


In [44]:
# Get summary stats for tuned model
eval_metrics1.describe()

,Accuracy,Precision,Recall,F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.927326,0.872532,0.927326,0.894777
std,0.088051,0.131044,0.088051,0.122974
min,0.590674,0.522878,0.590674,0.441091
25%,0.911727,0.831249,0.911727,0.869629
50%,0.954663,0.911382,0.954663,0.932521
75%,0.982208,0.964733,0.982208,0.973392
max,1.000000,1.000000,1.000000,1.000000


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

Again, the following code is very important to know how to build new gridseach parameters.

In [22]:
pipeline2.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__algorithm', 'clf__estimator__leaf_size', 'clf__estimator__metric', 'clf__estimator__metric_params', 'clf__estimator__n_jobs', 'clf__estimator__n_neighbors', 'clf__estimator__p', 'clf__estimator__weights', 'clf__estimator', 'clf__n_jobs'])

In [45]:
# Using SVM instead of Random Forest Classifier
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])
                 
parameters2 = {'clf__estimator__n_neighbors': [1,5],  'clf__estimator__leaf_size': [2,5], 'clf__estimator__p': [1,5]}

cv2 = GridSearchCV(pipeline2, param_grid = parameters2, scoring = scorer, verbose = 10)

# Find best parameters
np.random.seed(41)
tuned_model2 = cv2.fit(X_train, Y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5; 1/8] START clf__estimator__leaf_size=2, clf__estimator__n_neighbors=1, clf__estimator__p=1
[CV 1/5; 1/8] END clf__estimator__leaf_size=2, clf__estimator__n_neighbors=1, clf__estimator__p=1; total time= 1.5min
[CV 2/5; 1/8] START clf__estimator__leaf_size=2, clf__estimator__n_neighbors=1, clf__estimator__p=1
[CV 2/5; 1/8] END clf__estimator__leaf_size=2, clf__estimator__n_neighbors=1, clf__estimator__p=1; total time= 1.6min
[CV 3/5; 1/8] START clf__estimator__leaf_size=2, clf__estimator__n_neighbors=1, clf__estimator__p=1
[CV 3/5; 1/8] END clf__estimator__leaf_size=2, clf__estimator__n_neighbors=1, clf__estimator__p=1; total time= 1.9min
[CV 4/5; 1/8] START clf__estimator__leaf_size=2, clf__estimator__n_neighbors=1, clf__estimator__p=1
[CV 4/5; 1/8] END clf__estimator__leaf_size=2, clf__estimator__n_neighbors=1, clf__estimator__p=1; total time= 1.7min
[CV 5/5; 1/8] START clf__estimator__leaf_size=2, clf__estimator__n_n

[CV 3/5; 8/8] END clf__estimator__leaf_size=5, clf__estimator__n_neighbors=5, clf__estimator__p=5; total time=  15.0s
[CV 4/5; 8/8] START clf__estimator__leaf_size=5, clf__estimator__n_neighbors=5, clf__estimator__p=5
[CV 4/5; 8/8] END clf__estimator__leaf_size=5, clf__estimator__n_neighbors=5, clf__estimator__p=5; total time=  15.2s
[CV 5/5; 8/8] START clf__estimator__leaf_size=5, clf__estimator__n_neighbors=5, clf__estimator__p=5
[CV 5/5; 8/8] END clf__estimator__leaf_size=5, clf__estimator__n_neighbors=5, clf__estimator__p=5; total time=  14.7s


The task of improving the model is not easy, as the Random Forest classifier is an outstanding algorithm. The choice of the "KNeighborsClassifier()" mostly reflected that what was possible. Other algorithms, such as SVC, were very difficult to fit.

In [25]:
# Get results of grid search
tuned_model2.cv_results_

{'mean_fit_time': array([55.0037159 , 19.99312067, 17.07501431, 16.62494526, 14.65011358,
        14.59682717, 14.81111932, 14.25886297]),
 'std_fit_time': array([41.43601498,  2.28694115,  3.1185102 ,  3.09615973,  0.3363829 ,
         0.20105982,  0.83202633,  0.09936335]),
 'mean_score_time': array([167.07589002,   0.        , 132.39266534,   0.        ,
         97.24220662,   0.        , 104.96142077,   0.        ]),
 'std_score_time': array([ 7.62820091,  0.        , 18.4035491 ,  0.        ,  8.17844224,
         0.        ,  4.23586402,  0.        ]),
 'param_clf__estimator__leaf_size': masked_array(data=[2, 2, 2, 2, 5, 5, 5, 5],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_clf__estimator__n_neighbors': masked_array(data=[1, 1, 5, 5, 1, 1, 5, 5],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_clf__e

In [26]:
# Parameters for best mean test score
tuned_model2.best_params_

{'clf__estimator__leaf_size': 2,
 'clf__estimator__n_neighbors': 1,
 'clf__estimator__p': 1}

In [46]:
# Calculate evaluation metrics for test set
tuned_pred_test2 = tuned_model2.predict(X_test)

eval_metrics2 = get_eval_metrics(np.array(Y_test), tuned_pred_test2, col_names)

print(eval_metrics2)

                        Accuracy  Precision    Recall        F1
related                 0.135172   0.606956  0.135172  0.214316
request                 0.806461   0.751474  0.806461  0.768063
offer                   0.995733   0.992393  0.995733  0.994060
aid_related             0.574368   0.513044  0.574368  0.480135
medical_help            0.913593   0.861243  0.913593  0.885705
medical_products        0.940415   0.902019  0.940415  0.919985
search_and_rescue       0.972265   0.954869  0.972265  0.962196
security                0.978665   0.964935  0.978665  0.971532
military                0.967998   0.939927  0.967998  0.953756
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.925328   0.884180  0.925328  0.902427
food                    0.874581   0.817692  0.874581  0.839016
shelter                 0.896068   0.844758  0.896068  0.867021
clothing                0.981865   0.968401  0.981865  0.974516
money                   0.973331   0.956

From the above results it becomes clear that the Random Forest classifier tends to have a much better performance.

### 9. Export your model as a pickle file

In [ ]:
# Pickle best model
pickle.dump(tuned_model, open('disaster_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.